<a href="https://colab.research.google.com/github/ahatasham5/Bangla_Handwritting_CSE366/blob/main/CNN_Bangla_Handwritting_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is the notebook you have seen in the overview video.

## Creating the CNN model

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

In [ ]:
import zipfile
from google.colab import drive
# Mount your Google Drive
drive.mount('/content/drive')

# Replace '/content/drive/MyDrive/archive2.zip' with the actual path to your zip file on Google Drive
with zipfile.ZipFile('/content/drive/MyDrive/HWrec/archive3.zip', 'r') as zip_ref:
    # Extract all the contents of the zip file in the current directory
    zip_ref.extractall()

Mounted at /content/drive


In [ ]:
!pwd


/content


In [ ]:
classifier = Sequential()

classifier.add(Conv2D(filters = 128, kernel_size = (3, 3), activation = 'relu', input_shape = (40, 40, 3)))
classifier.add(MaxPooling2D(pool_size = (2, 2)))
classifier.add(Dropout(.2))

classifier.add(Conv2D(filters = 64, kernel_size = (3, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))
classifier.add(Dropout(.2))

classifier.add(Flatten())

classifier.add(Dense(units = 128, activation = 'relu'))
classifier.add(Dropout(.2))

classifier.add(Dense(units = 10, activation = 'softmax'))

classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

## Fitting the CNN model to the Dataset

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from PIL import ImageFile

In [ ]:
!pwd
!ls
!ls /content/archive3/Train

/content
archive3  drive  sample_data
0  1  2  3  4  5  6  7	8  9


Data Generate

In [ ]:
ImageFile.LOAD_TRUNCATED_IMAGES = True

train_datagen = ImageDataGenerator(rescale = 1./255, shear_range = .2, rotation_range = 25)
test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('/content/archive3/Test', target_size = (40, 40),
                                                 batch_size = 32, class_mode = 'categorical')
test_set = test_datagen.flow_from_directory('/content/archive3/Train', target_size = (40, 40),
                                                 batch_size = 32, class_mode = 'categorical')

Found 14724 images belonging to 10 classes.
Found 1120 images belonging to 10 classes.


In [ ]:
import numpy as np

# Get a single batch of images and labels
images_batch, labels_batch = next(training_set)

# Check if there are any NaN or Inf values in the images
if np.isnan(images_batch).any():
    print('Images contain NaN values.')
if np.isinf(images_batch).any():
    print('Images contain Inf values.')

In [ ]:
# Get a single batch of images and labels
images_batch, labels_batch = next(training_set)

# Print the shape of the images
print('Shape of images:', images_batch.shape)


Shape of images: (32, 40, 40, 3)


In [ ]:
from keras.callbacks import EarlyStopping

# Define the number of times to repeat the data generator
num_epochs = 3
num_repeats = 1  # Adjust as needed

early_stopping = EarlyStopping(monitor='val_loss', patience=5)

for _ in range(num_epochs):
    for _ in range(num_repeats):
        # Reset the data generator before each repeat
        training_set.reset()

        # Train for one epoch
        classifier.fit(training_set, steps_per_epoch=len(training_set), epochs=1, validation_data=test_set,
                       validation_steps=len(test_set), callbacks=[early_stopping])


461/461 [==============================] - 124s 269ms/step - loss: 0.2120 - accuracy: 0.9283 - val_loss: 0.1253 - val_accuracy: 0.9589


In [ ]:
classifier.fit(training_set, steps_per_epoch = 35, epochs = 20,
               validation_data = test_set, validation_steps = 460)


Epoch 1/20
14/35 [===========>..................] - ETA: 4s - loss: 0.4468 - accuracy: 0.8728

KeyboardInterrupt: ignored

In [ ]:
classifier.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 38, 38, 128)       3584      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 19, 19, 128)      0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 19, 19, 128)       0         
                                                                 
 conv2d_1 (Conv2D)           (None, 17, 17, 64)        73792     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 8, 8, 64)         0         
 2D)                                                             
                                                                 
 dropout_1 (Dropout)         (None, 8, 8, 64)          0

In [ ]:
!pwd
!ls /content/archive3/

/content
SinglePrediction  Test	Train


## Creating a graphical user interface to draw the character

In [ ]:
!pip install keras

In [ ]:
from IPython.display import display, HTML, Javascript
import io
import base64
from PIL import Image
import numpy as np
from tensorflow.keras.preprocessing import image
import ipywidgets as widgets




# Create an HTML canvas
canvas_html = """
<canvas width="400" height="400" id="drawing-canvas" style="border:1px solid black;"></canvas>
"""

# Create JavaScript code to handle mouse events and update the canvas
javascript = """
<script type="text/javascript">
    // Get the canvas element and its context
    var canvas = document.getElementById('drawing-canvas');
    var ctx = canvas.getContext('2d');

    // Set the stroke style
    ctx.strokeStyle = 'black';
    ctx.lineWidth = 10;

    // Flag to indicate whether the mouse button is pressed
    var isDrawing = false;

    // Function to handle mouse down events
    function handleMouseDown(event) {
        // Start drawing
        isDrawing = true;
        ctx.beginPath();
        ctx.moveTo(event.offsetX, event.offsetY);
    }

    // Function to handle mouse move events
    function handleMouseMove(event) {
        if (isDrawing) {
            // Draw a line to the current mouse position
            ctx.lineTo(event.offsetX, event.offsetY);
            ctx.stroke();
        }
    }

    // Function to handle mouse up events
    function handleMouseUp(event) {
        // Stop drawing
        isDrawing = false;
    }

    // Register event listeners
    canvas.addEventListener('mousedown', handleMouseDown);
    canvas.addEventListener('mousemove', handleMouseMove);
    canvas.addEventListener('mouseup', handleMouseUp);
</script>
"""

# Display the canvas and run the JavaScript code
display(HTML(canvas_html + javascript))

def determine_character(res):
    if res == 0:
        print('prediction : 0')
    elif res == 1:
        print('prediction : 1')
    elif res == 2:
        print('prediction : 2')
    elif res == 3:
        print('prediction : 3')
    elif res == 4:
        print('prediction : 4')
    elif res == 5:
        print('prediction : 5')
    elif res == 6:
        print('prediction : 6')
    elif res == 7:
        print('prediction : 7')
    elif res == 8:
        print('prediction : 8')
    elif res == 9:
        print('prediction : 9')

def single_prediction(test_img):
    test_img_arr = image.img_to_array(test_img)
    test_img_arr = np.expand_dims(test_img_arr, axis = 0)
    prediction = classifier.predict(test_img_arr)
    result = np.argmax(prediction, axis = 1)
    determine_character(result)

from google.colab import output

def predict_from_canvas():
    # Define a JavaScript function to get the image data from the canvas
    get_image_data_js = """
        async function getImageData() {
            var canvas = document.getElementById('drawing-canvas');
            return canvas.toDataURL();
        }
        getImageData();
    """

    # Execute the JavaScript code and get the result
    data = output.eval_js(get_image_data_js)

    # Convert the image data to a PIL Image
    img = Image.open(io.BytesIO(base64.b64decode(data.split(',')[1])))

    # Convert the image to RGB format
    img = img.convert('RGB')

    # Resize the image to the desired size
    img = img.resize((40, 40))

    # Make a prediction using the trained model
    single_prediction(img)

    # Create a button to make a prediction
predict_button = widgets.Button(description='Predict')

# Function to handle button click
def on_predict_button_click(b):
    predict_from_canvas()

# Register event handler
predict_button.on_click(on_predict_button_click)

# Display the button
display(predict_button)

Button(description='Predict', style=ButtonStyle())

1/1 [==============================] - 0s 35ms/step
prediction : 4
